In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)


train_data, validation_data, test_data = tfds.load( name = "imdb_reviews", split = [ 'train', 'test[:40%]', 'test[40%:]'], as_supervised=True )

print(train_data)


embed = hub.load("https://kaggle.com/models/google/gnews-swivel/frameworks/TensorFlow2/variations/tf2-preview-20dim/versions/1")



hub_layer = hub.KerasLayer("https://kaggle.com/models/google/gnews-swivel/frameworks/TensorFlow2/variations/tf2-preview-20dim/versions/1", output_shape=[20],
                           input_shape=[], dtype=tf.string, trainable = True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()




2025-03-31 16:37:57.693728: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 16:37:57.694760: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-31 16:37:57.697960: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-31 16:37:57.706532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743419277.721692   21561 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743419277.72

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7c9d208b1690> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [2]:
import numpy as np
import tensorflow_hub as hub

import logging
import tensorflow as tf

tf.get_logger().setLevel(logging.ERROR)  # Suppresses INFO and WARNING messages


class HubLayerWrapper(tf.keras.layers.Layer):
    def __init__(self, hub_url, trainable=True, **kwargs):
        super(HubLayerWrapper, self).__init__(**kwargs)
        self.hub_layer = hub.KerasLayer(hub_url, trainable=trainable)
    def call(self, inputs):
        return self.hub_layer(inputs)

inputs = tf.keras.Input(shape=(), dtype=tf.string)
embeddings = HubLayerWrapper("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1", trainable=True)(inputs)
dense1 = tf.keras.layers.Dense(16, activation='relu')(embeddings)
dense2 = tf.keras.layers.Dense(8, activation='relu')(dense1)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hub_layer_wrapper_1             │ (None, 20)             │             0 │
│ (HubLayerWrapper)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 481 (1.88 KB)

 Trainable params: 481 (1.88 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import logging
tf.get_logger().setLevel(logging.CRITICAL)


train_data, validation_data, test_data = tfds.load( name="imdb_reviews", split=['train[:60%]', 'train[60%:]', 'test'], as_supervised=True)
print(train_data)

train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape = [], dtype = tf.string, trainable=True)

print(hub_layer(train_example_batch[:3]))

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

model.compile( optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy( from_logits=True), metrics=['accuracy'])

history = model.fit(train_data.shuffle(10000).batch(100), epochs = 25, validation_data = validation_data.batch(100), verbose=1)


<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
tf.Tensor(
[[ 1.765786   -3.882232    3.9134233  -1.5557289  -3.3362343  -1.7357955
  -1.9954445   1.2989551   5.081598   -1.1041286  -2.0503852  -0.72675157
  -0.65675956  0.24436149 -3.7208383   2.0954835   2.2969332  -2.0689783
  -2.9489717  -1.1315987 ]
 [ 1.8804485  -2.5852382   3.4066997   1.0982676  -4.056685   -4.891284
  -2.785554    1.3874227   3.8476458  -0.9256538  -1.896706    1.2113281
   0.11474707  0.76209456 -4.8791065   2.906149    4.7087674  -2.3652055
  -3.5015898  -1.6390051 ]
 [ 0.71152234 -0.6353217   1.7385626  -1.1168286  -0.5451594  -1.1808156
   0.09504455  1.4653089   0.66059524  0.79308075 -2.2268345   0.07446612
  -1.4075904  -0.70645386 -1.907037    1.4419787   1.9551861  -0.42660055
  -2.8022065   0.43727064]], shape=(3, 20), dtype=float32)


2025-04-01 19:19:25.898719: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7b2935aaee10> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [10]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.get_logger().setLevel('ERROR')

# Load the IMDb dataset splits
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=['train[:60%]', 'train[60%:]', 'test'],
    as_supervised=True
)

# Retrieve a batch for testing
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

# Define a custom layer to wrap the Hub layer
class HubLayerWrapper(tf.keras.layers.Layer):
    def __init__(self, hub_url, **kwargs):
        super(HubLayerWrapper, self).__init__(**kwargs)
        self.hub_layer = hub.KerasLayer(
            hub_url,
            trainable=True,
            input_shape=[],  # expects a 1D tensor of strings
            dtype=tf.string
        )
    def call(self, inputs):
        return self.hub_layer(inputs)

embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
custom_hub_layer = HubLayerWrapper(embedding_url)

# Test the custom hub layer on the first 3 examples of the batch
print(custom_hub_layer(train_example_batch[:3]))

# Build the model using the Functional API
inputs = tf.keras.Input(shape=(), dtype=tf.string)
x = custom_hub_layer(inputs)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


model.compile( optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy( from_logits=True), metrics=['accuracy'])

history = model.fit(train_data.shuffle(10000).batch(100), epochs = 25, validation_data = validation_data.batch(100), verbose=1)

results = model.evaluate(test_data.batch(100), verbose=2)
for name,value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name,value))


tf.Tensor(
[[ 1.765786   -3.882232    3.9134233  -1.5557289  -3.3362343  -1.7357955
  -1.9954445   1.2989551   5.081598   -1.1041286  -2.0503852  -0.72675157
  -0.65675956  0.24436149 -3.7208383   2.0954835   2.2969332  -2.0689783
  -2.9489717  -1.1315987 ]
 [ 1.8804485  -2.5852382   3.4066997   1.0982676  -4.056685   -4.891284
  -2.785554    1.3874227   3.8476458  -0.9256538  -1.896706    1.2113281
   0.11474707  0.76209456 -4.8791065   2.906149    4.7087674  -2.3652055
  -3.5015898  -1.6390051 ]
 [ 0.71152234 -0.6353217   1.7385626  -1.1168286  -0.5451594  -1.1808156
   0.09504455  1.4653089   0.66059524  0.79308075 -2.2268345   0.07446612
  -1.4075904  -0.70645386 -1.907037    1.4419787   1.9551861  -0.42660055
  -2.8022065   0.43727064]], shape=(3, 20), dtype=float32)


2025-04-01 20:09:54.752495: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hub_layer_wrapper_2             │ (None, 20)             │             0 │
│ (HubLayerWrapper)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 353 (1.38 KB)

 Trainable params: 353 (1.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5860 - loss: 0.7227 - val_accuracy: 0.6503 - val_loss: 0.6297
Epoch 2/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6512 - loss: 0.6243 - val_accuracy: 0.6758 - val_loss: 0.6003
Epoch 3/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6721 - loss: 0.6016 - val_accuracy: 0.6912 - val_loss: 0.5857
Epoch 4/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6843 - loss: 0.5893 - val_accuracy: 0.6997 - val_loss: 0.5780
Epoch 5/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7042 - loss: 0.5737 - val_accuracy: 0.7073 - val_loss: 0.5718
Epoch 6/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7019 - loss: 0.5766 - val_accuracy: 0.7094 - val_loss: 0.5675
Epoch 7/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7037 - loss: 0.5681 - val_accuracy: 0.7138 - val_loss: 0.5636
Epoch 8/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7052 - loss: 0.5704 - val_accuracy: 0

In [1]:
import pandas as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import PIL.Image, PIL.ImageFont, PIL.ImageDraw

im_width = 75
im_height = 75
use_normalised_coordinates = True

def draw_bounding_boxes_on_image_array( image, boxes, color=[], thickness=1, display_str_list=()):
    imagge_pil = PIL.Image.fromarray(image)
    rgbimg = PIL.Image.new("RGBA", imagge_pil.size)
    rgbimg.paste(imagge_pil)

    draw_bounding_boxes_on_image( image, boxes, color=[], thickness=1, display_str_list=() )
    return np.array(rgbimg)

def draw_bounding_boxes_on_image( image, boxes, color=[], thickness=1, display_str_list=() )
    boxes_shape = boxes.shape
if not boxes_shape:
    return


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Set up the figure and axis
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
ax.grid(True)
ax.set_title('Rotating Magnetic Field (3-Phase)')

# Initialize arrows for R, Y, B phases and resultant
r_arrow = ax.arrow(0, 0, 0, 0, color='red', width=0.02)
y_arrow = ax.arrow(0, 0, 0, 0, color='yellow', width=0.02)
b_arrow = ax.arrow(0, 0, 0, 0, color='blue', width=0.02)
resultant_arrow = ax.arrow(0, 0, 0, 0, color='green', width=0.04)

# Function to update arrows per frame
def update(frame):
    ax.patches.clear()
    omega_t = np.radians(frame)

    # Phase angles
    r = np.sin(omega_t) * np.array([np.cos(0), np.sin(0)])
    y = np.sin(omega_t - 2 * np.pi / 3) * np.array([np.cos(2 * np.pi / 3), np.sin(2 * np.pi / 3)])
    b = np.sin(omega_t - 4 * np.pi / 3) * np.array([np.cos(4 * np.pi / 3), np.sin(4 * np.pi / 3)])

    resultant = r + y + b

    ax.arrow(0, 0, r[0], r[1], color='red', width=0.02)
    ax.arrow(0, 0, y[0], y[1], color='yellow', width=0.02)
    ax.arrow(0, 0, b[0], b[1], color='blue', width=0.02)
    ax.arrow(0, 0, resultant[0], resultant[1], color='green', width=0.04)

# Create the animation
ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 5), interval=100)

plt.close()  # Prevent duplicate display in notebook
ani


In [ ]:
import pandas as np
import numpy as np

np.random.seed(0)

n=400

'Pregnancies': np.random.randint(0, 15, size=n)
'Glucose': np.random.randint()


In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)  
n = 400

data = {
    'Pregnancies': np.random.randint(0, 15, size=n),
    'Glucose': np.random.normal(120, 30, size=n).astype(int),
    'BloodPressure': np.random.normal(70, 12, size=n).astype(int),
    'SkinThickness': np.random.normal(20, 10, size=n).astype(int),
    'Insulin': np.random.normal(80, 45, size=n).astype(int),
    'BMI': np.round(np.random.normal(30, 7, size=n), 1),
    'DiabetesPedigreeFunction': np.round(np.random.normal(0.5, 0.2, size=n), 3),
    'Age': np.random.randint(20, 70, size=n),
}

data['Outcome'] = (
    (data['Glucose'] > 130).astype(int) +
    (data['BMI'] > 32).astype(int) +
    (data['Age'] > 40).astype(int)
)
data['Outcome'] = (data['Outcome'] >= 2).astype(int)

df = pd.DataFrame(data)
df.to_csv("diabetes_synthetic.csv", index=False)
print("Synthetic data saved!")


In [ ]:
import pandas as pd
import numpy as np

n = 400

data = (
    'Pregnancies': np.random.randint(0, 10, size=n)
    'Glucose': np.random.normal( 100, 25, size=n ).astype(int)
    'BP': np.random.normal( 100, 20, size=n ).astype(int)
    'SkinThickness': np.random.normal( 30, 5, size=n).astype(int)
    'Insulin': np.random.normal( 120, 40, size=n ).astype(int)
    'B.M.I': np.random.normal(  ).astype(int)


)
